In [1]:
from census import Census
from dotenv import load_dotenv
from bson import ObjectId

In [2]:
from pymongo import MongoClient
client = MongoClient()
db = client.get_database('companies_project')

In [3]:
load_dotenv()
census_api_k = os.getenv('census_api_key')
c = Census(census_api_k, year=2018)

In [14]:
offices = list(db.companies_usa_filt1.find({'offices.demographics.school%':{'$exists':False}}, {'_id':1, 'offices.zip_code':1}))

In [11]:
def schooling(finded):
    counter = 0
    for comp in finded:
        id = ObjectId(comp['_id'])
        zip_code = comp['offices']['zip_code'][:5]
        try:
            schoolP = {'kinder_enrollP': c.acs5dp.zipcode('DP02_0054PE', zip_code)[0]['DP02_0054PE'], 
                       'presch_enrollP': c.acs5dp.zipcode('DP02_0053PE', zip_code)[0]['DP02_0053PE'], 
                       'element_enrollP': c.acs5dp.zipcode('DP02_0055PE', zip_code)[0]['DP02_0055PE']}
            educ_lvlP = {'highschP': c.acs5dp.zipcode('DP02_0066PE', zip_code)[0]['DP02_0066PE'],
                         'bachelP': c.acs5dp.zipcode('DP02_0067PE', zip_code)[0]['DP02_0067PE']} 
            db.companies_usa_filt1.update_one({'_id':id}, {'$set':{'offices.demographics.school%': schoolP}})
            db.companies_usa_filt1.update_one({'_id':id}, {'$set':{'offices.demographics.educ_lvl%': educ_lvlP}})
        except:
            counter += 1
    return f'Failed to append schooling in {counter} documents'

In [15]:
schooling(offices)

'Failed to append schooling in 0 documents'